<a href="https://colab.research.google.com/github/gopal2812/mlblr/blob/master/assignment6_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import sys
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from random import randint


Using TensorFlow backend.


In [2]:
!ls

sample_data


In [3]:
from google.colab import files
uploaded = files.upload()

Saving wonderland.txt to wonderland.txt


In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

2. Remove all the punctuation from the source text

In [0]:
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
  
 	# replace '\n' with a space ' '
	doc = doc.replace('\n', ' ')
  
	# split into tokens by full stop
	tokens = doc.split('.')
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [0]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [7]:
in_filename = "wonderland.txt"
doc = load_doc(in_filename)
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['           alices adventures in wonderland  lewis carroll  the millennium fulcrum edition 3', '0     chapter i', ' down the rabbithole  alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations  so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her', '  there was nothing so very remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself oh dear oh dear i shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all s

3. Train the model on padded sequences (Links to an external site.) rather than random sequences of characters. 

In [8]:
#We would be taking a compromise so we save a lot of memory and have to truncate maybe 5% of all the sequences. 
#Sequences that are too long should be truncated at the end
num_lines = [len(line) for line in tokens]
std_line = np.std(num_lines)
mean_line = np.mean(num_lines)
seq_len= (int)((mean_line+2*std_line))
print(mean_line, std_line, seq_len, np.median(num_lines))

#also find the most frequent length
(values,counts) = np.unique(num_lines,return_counts=True)
ind=np.argmax(counts)
print(values[ind])

tokenizer = Tokenizer()


def get_sequence_of_tokens(tokens):
    ## tokenization
    tokenizer.fit_on_texts(tokens)
    vocab_size = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in tokens:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, vocab_size

sequences, vocab_size = get_sequence_of_tokens(tokens)
sequences[:1]




137.6024217961655 135.70159788002653 409 94.0
30


[[303, 467]]

In [0]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(i) for i in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    X, y = input_sequences[:,:-1],input_sequences[:,-1]
    y = to_categorical(y, num_classes=vocab_size)
    return X, y, max_sequence_len

X, y, seq_length = generate_padded_sequences(sequences)

5. Add dropout to the input layer, remove it from the layer before dense layer. Use Dropout value of 0.1 everywhere

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 30, input_length=seq_length-1))
#model.add(Dropout(0.1))
#A dropout on the input means that for a given probability, the data on the input 
# connection to each LSTM block will be excluded from node activation and weight updates.
model.add(LSTM(256, input_shape=(seq_length, vocab_size), return_sequences=True, dropout=0.1))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 30)           79440     
_________________________________________________________________
dropout_5 (Dropout)          (None, 300, 30)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 300, 256)          293888    
_________________________________________________________________
dropout_6 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 2648)              680536    
Total params: 1,579,176
Trainable params: 1,579,176
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

4. Train the model for 100 epochs

In [15]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

W0726 17:37:04.334365 140093314234240 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
25562/25562 [==============================] - 188s 7ms/step - loss: 6.4132

Epoch 00001: loss improved from inf to 6.41316, saving model to weights-improvement-01-6.4132-bigger.hdf5
Epoch 2/100
25562/25562 [==============================] - 182s 7ms/step - loss: 6.1998

Epoch 00002: loss improved from 6.41316 to 6.19985, saving model to weights-improvement-02-6.1998-bigger.hdf5
Epoch 3/100
25562/25562 [==============================] - 183s 7ms/step - loss: 6.1960

Epoch 00003: loss improved from 6.19985 to 6.19595, saving model to weights-improvement-03-6.1960-bigger.hdf5
Epoch 4/100
25562/25562 [==============================] - 183s 7ms/step - loss: 6.1988

Epoch 00004: loss did not improve from 6.19595
Epoch 5/100
25562/25562 [==============================] - 182s 7ms/step - loss: 6.2020

Epoch 00005: loss did not improve from 6.19595
Epoch 6/100
25562/25562 [==============================] - 182s 7ms/step - loss: 6.1969

Epoch 00006: loss did not improve from 6.19595

In [16]:
!ls

sample_data
weights-improvement-01-6.4132-bigger.hdf5
weights-improvement-02-6.1998-bigger.hdf5
weights-improvement-03-6.1960-bigger.hdf5
weights-improvement-08-6.1950-bigger.hdf5
weights-improvement-18-6.1946-bigger.hdf5
weights-improvement-37-6.1936-bigger.hdf5
weights-improvement-40-6.1930-bigger.hdf5
weights-improvement-41-6.1926-bigger.hdf5
weights-improvement-42-6.1922-bigger.hdf5
weights-improvement-48-6.1896-bigger.hdf5
wonderland.txt


In [0]:
predict_len=seq_len

1. Predict 500 characters only

In [0]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded],
                            maxlen=seq_length,
                            padding='pre',
                            truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		#print(yhat.shape, yhat)
    # map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			#print("Index:", index)
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		if(len(in_text) >= 500):
				print("completed 500 character")
				break
		result.append(out_word)
	return ' '.join(result)

In [19]:
print(tokens[0])
# select a seed text
seed_text = tokens[randint(0,len(tokens))]
print(seed_text + '\n')

           alices adventures in wonderland  lewis carroll  the millennium fulcrum edition 3
 now if you only kept on good terms with him hed do almost anything you liked with the clock



In [20]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length-1, seed_text, 500)
print(generated)

completed 500 character
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
